In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("Decision_tree_data.csv")
df.head()

,age,gender,occupation,education_level,marital_status,income,credit_score,loan_status
0,32,Male,Engineer,Bachelor's,Married,85000,720,Approved
1,45,Female,Teacher,Master's,Single,62000,680,Approved
2,28,Male,Student,High School,Single,25000,590,Denied
3,51,Female,Manager,Bachelor's,Married,105000,780,Approved
4,36,Male,Accountant,Bachelor's,Married,75000,710,Approved


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              61 non-null     int64 
 1   gender           61 non-null     object
 2   occupation       61 non-null     object
 3   education_level  61 non-null     object
 4   marital_status   61 non-null     object
 5   income           61 non-null     int64 
 6   credit_score     61 non-null     int64 
 7   loan_status      61 non-null     object
dtypes: int64(3), object(5)
memory usage: 3.9+ KB


In [6]:
cat_col_label=["loan_status"]
one_col_hot=["gender","occupation","education_level","marital_status"]
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
df[cat_col_label[0]]=lb.fit_transform(df[cat_col_label[0]])

In [7]:
df=pd.get_dummies(df,columns=one_col_hot,drop_first=True)

In [8]:
df

,age,income,credit_score,loan_status,gender_Male,occupation_Analyst,occupation_Architect,occupation_Artist,occupation_Banker,occupation_Chef,occupation_Consultant,occupation_Dentist,occupation_Designer,occupation_Doctor,occupation_Editor,occupation_Electrician,occupation_Engineer,occupation_HR,occupation_IT,occupation_Lawyer,occupation_Manager,occupation_Marketing,occupation_Musician,occupation_Nurse,occupation_Pharmacist,occupation_Photographer,occupation_Pilot,occupation_Professor,occupation_Programmer,occupation_Psychologist,occupation_Realtor,occupation_Receptionist,occupation_Researcher,occupation_Sales,occupation_Salesman,occupation_Server,occupation_Software,occupation_Student,occupation_Stylist,occupation_Teacher,occupation_Veterinarian,occupation_Writer,education_level_Bachelor's,education_level_Doctoral,education_level_High School,education_level_Master's,marital_status_Single
0,32,85000,720,0,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
1,45,62000,680,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True
2,28,25000,590,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,True
3,51,105000,780,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
4,36,75000,710,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,39,100000,770,0,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
57,25,32000,570,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True
58,43,95000,760,0,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
59,30,55000,650,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True


In [9]:
df.isna().sum()

age                            0
income                         0
credit_score                   0
loan_status                    0
gender_Male                    0
occupation_Analyst             0
occupation_Architect           0
occupation_Artist              0
occupation_Banker              0
occupation_Chef                0
occupation_Consultant          0
occupation_Dentist             0
occupation_Designer            0
occupation_Doctor              0
occupation_Editor              0
occupation_Electrician         0
occupation_Engineer            0
occupation_HR                  0
occupation_IT                  0
occupation_Lawyer              0
occupation_Manager             0
occupation_Marketing           0
occupation_Musician            0
occupation_Nurse               0
occupation_Pharmacist          0
occupation_Photographer        0
occupation_Pilot               0
occupation_Professor           0
occupation_Programmer          0
occupation_Psychologist        0
occupation

In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
X=df.drop(columns=["loan_status"],axis=1)
y=df["loan_status"]

In [14]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,random_state=42,test_size=0.2,stratify=y)

In [19]:
Xtrain.shape,Xtest.shape,ytrain.shape,ytest.shape

((48, 46), (13, 46), (48,), (13,))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
DC=DecisionTreeClassifier(random_state=42)
test_parm={
    "criterion":['gini', 'entropy', 'log_loss'],
    "splitter": ['best', 'random'],
    "max_depth":[5,7,10,12,15],
    "min_samples_split":[5,7,10,12],
    "min_samples_leaf": [5,7,10,12],
}
GS=GridSearchCV(DC,test_parm,cv=5, n_jobs=-1, scoring='accuracy')

In [15]:
GS.fit(Xtrain,ytrain)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'criterion': ['gini', 'entropy', ...], 'max_depth': [5, 7, ...], 'min_samples_leaf': [5, 7, ...], 'min_samples_split': [5, 7, ...], ...}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [16]:
GS.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'min_samples_leaf': 5,
 'min_samples_split': 5,
 'splitter': 'best'}

In [18]:
DC=DecisionTreeClassifier(criterion='gini',max_depth=5,min_samples_leaf=5,min_samples_split=5,splitter='best')

In [20]:
DC.fit(Xtrain,ytrain)

,criterion,'gini'
,splitter,'best'
,max_depth,5
,min_samples_split,5
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [21]:
pred_dc=DC.predict(Xtest)


In [24]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(pred_dc,ytest))
print(classification_report(pred_dc,ytest))

[[10  0]
 [ 0  3]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         3

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13

